In [ ]:
import matplotlib.pyplot as plt

import torch
from torchsummary import summary

import numpy as np
import sys, argparse, os

import sys
sys.path.insert(0, '../')
sys.path.append("/scratch2/flood_sim2/mlflood/")
from pathlib import Path
from mlflood.conf import PATH_DATA
from mlflood.conf import rain_const, waterdepth_diff_const, max_depth_const_dict

import torch.nn.functional as F

from mlflood.utils import new_log
from models.utae import UTAE
from models.CNNrolling import CNNrolling
from models.unet3d import UNet3D
from models.unet import UNet
from models.unet_bay import UNet_bay

#from dataloading.dataset_max_multi import load_test_dataset
#from dataloading.dataset_max_multi import dataloader_args_test

# from dataloading.dataset_max_709_stef import load_test_dataset
# from dataloading.dataset_max_709_stef import dataloader_args_test

#from dataloading.dataset_max_lu import load_test_dataset
#from dataloading.dataset_max_lu import dataloader_args_test

from dataloading.dataset_max_709 import load_test_dataset
from dataloading.dataset_max_709 import dataloader_args_test

from training import *

In [ ]:
#path_exp = "/scratch2/ml_flood/data/checkpoints/709/cluster/unet_l1/experiment_0/"
path_exp = "/scratch2/flood_sim2/data/checkpoints/709/cluster/unet_bay_new3/experiment_0/"
#path_exp = "/scratch2/flood_sim2/data/checkpoints/multi/cluster/unet_ens/experiment_0/"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', device)
args = []   

In [ ]:
## For Stefania
import yaml 
parser = argparse.ArgumentParser(description="evaluation")
parser.add_argument("--catchment_kwargs", default='./default_catchment_kwargs.yml', type=str, 
                    help="path to catchment kwargs saved in yml file")

exp_hp = ['--catchment_kwargs=../mlflood/exp_yml/exp_unet_no_bay_10.yml']
args = parser.parse_args(exp_hp)

with open(args.catchment_kwargs) as file:
    catchment_kwargs = yaml.full_load(file)
    
# Always in eval mode
catchment_kwargs['fix_indexes'] = True 
catchment_kwargs['use_diff_dem'] = True

In [ ]:
parser = argparse.ArgumentParser(description="evaluation")
parser.add_argument("--task", default='max_depth', type=str, 
                    help="task to run")

exp_hp = ['--task=max_depth']
args = parser.parse_args(exp_hp)


In [ ]:
# ## For Priyanka

# ### Catchment settings
catchment_kwargs = {}
catchment_kwargs["num"] = "709_max" #"luzern_max"
catchment_kwargs["tau"] = 0.5
catchment_kwargs["timestep"]= 1     # for timestep >1 use CNN rolling or Unet
catchment_kwargs["sample_type"]="single"
catchment_kwargs["dim_patch"]=256
catchment_kwargs["fix_indexes"]=True
catchment_kwargs["border_size"] = 0
catchment_kwargs["normalize_output"] = False
catchment_kwargs["use_diff_dem"] = True
catchment_kwargs["num_patch"] = 10      # number of patches to generate from a timestep
catchment_kwargs["predict_ahead"] = 5
use_mask = catchment_kwargs["use_mask_feat"] = False
catchment_kwargs["use_feat"] =True
catchment_kwargs["ts_out"] = None

catchment_num = catchment_kwargs["num"]

In [ ]:
dataloaders = {}
dataset = load_test_dataset(catchment_kwargs)
                                                             
dataloaders["test"] = dataloader_args_test(dataset, catchment_num = catchment_kwargs['num'])
dataset_test = dataloaders["test"]

In [ ]:
model = UNet_bay(args, catchment_kwargs)  
file_path1 = path_exp + "ensemble_model1.pth.tar"
#file_path1 = path_exp + "model.pth.tar"
model.load_state_dict(torch.load(file_path1))
model.cuda()

In [ ]:
folder_loc = path_exp + "results/plots/"
print(folder_loc)
h51 = catchment_kwargs["num"].strip() + "_"

In [ ]:
from evaluation_max_bay import predict_event
event_num = 1
# timestep = 12
pred_cnn, gt_cnn, mask_cnn, sigma_cnn = predict_event(model, dataset, event_num, 'unet', start_ts=0, ar = False, T = None)

# View reconstruction

In [ ]:
sigma_cnn 

In [ ]:
recons_mask_full = dataset.dem_mask
import matplotlib
ev = 0
for ev in range(1):
    
    pred =  pred_cnn*mask_cnn#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
    pred = pred*max_depth_const_dict[catchment_num]#[1000:1300, 2000:2300]
    gt = gt_cnn*max_depth_const_dict[catchment_num]#[1000:1300, 2000:2300]#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
    
#     pred =  recons_pred_full*recons_mask_full#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
#     pred = pred.cpu().detach().numpy()
#     pred = pred [1000:1300, 2000:2300]
#     gt = recons_gt_full.cpu().detach().numpy()#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
#     gt = gt[1000:1300, 2000:2300]

    max_v = np.max(pred)
    min_v = np.min(pred)

    if np.max(gt) > max_v:
        max_v = np.max(gt)
    
    l = []
    l.append(gt)
    l.append(pred)

    title = []
    title.append('Ground truth')
    title.append('Prediction')

    i = 0
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,16))
    for ax in axes.flat:
        ax.grid(False)
        ax.title.set_text(title[i])
        im = ax.imshow(l[i], cmap='GnBu', norm=matplotlib.colors.LogNorm(vmin=0.0001, vmax=max_v))
        i = i + 1

    fig.subplots_adjust(right=0.8)
    fig.colorbar(im, ax=ax, extend='both', fraction=0.036, pad=0.02)

#     figtitle = "unet, 500 epochs, event=" + str(event_num)
#     plt.suptitle(figtitle, y=0.86)

    filename = folder_loc + str(h51) + 'pred_gt_' + str(event_num)+ '.png'
    #filename = folder_loc + 'test'+ '.png'
    plt.savefig(filename, dpi=1200)

    plt.show()

In [ ]:
recons_mask_full = dataset.dem_mask
import matplotlib
ev = 0
b = np.sqrt(2)
for ev in range(1):
    
    sigma = sigma_cnn*mask_cnn#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
    pred = pred_cnn*mask_cnn#[1000:1300, 2000:2300]

    pred = pred*max_depth_const_dict[catchment_num]
    gt = gt_cnn*max_depth_const_dict[catchment_num]#[1000:1300, 2000:2300]#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
    sigma = sigma*max_depth_const_dict[catchment_num]
    res = np.abs(pred - gt_cnn)
    
    
    sigma = sigma#[1000:1100,1000:1100]
    res = res#[1000:1100,1000:1100]
    
#     pred =  recons_pred_full*recons_mask_full#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
#     pred = pred.cpu().detach().numpy()
#     pred = pred [1000:1300, 2000:2300]
#     gt = recons_gt_full.cpu().detach().numpy()#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
#     gt = gt[1000:1300, 2000:2300]

    max_v = np.max(sigma)
    min_v = np.min(sigma)

    if np.max(res) > max_v:
        max_v = np.max(res)
    
    l = []
    l.append(sigma)
    l.append(res)

    title = []
    title.append('Uncertainty')
    title.append('Residuals')

    i = 0
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,16))
    for ax in axes.flat:
        ax.grid(False)
        ax.title.set_text(title[i])
        im = ax.imshow(l[i], cmap='Oranges', norm=matplotlib.colors.LogNorm(vmin=0.0001, vmax=max_v))
        i = i + 1

    fig.subplots_adjust(right=0.8)
    fig.colorbar(im, ax=ax, extend='both', fraction=0.036, pad=0.02)

#     figtitle = "unet, 500 epochs, event=" + str(event_num)
#     plt.suptitle(figtitle, y=0.86)

    filename = folder_loc + str(h51) + '_unc_pred_' + str(event_num)+ '.png'
    plt.savefig(filename, dpi=1200)

    plt.show()

In [ ]:
print(np.max(pred_cnn))
print(np.min(pred_cnn))

In [ ]:
print(np.max(res))
print(np.min(res))

In [ ]:
print(np.max(sigma))
print(np.min(sigma))

In [ ]:
mask_catc = mask_cnn.astype(bool)

In [ ]:
pred_vec = pred_cnn[mask_catc].flatten()
gt_vec = gt_cnn[mask_catc].flatten()

pred_vec = pred_vec*max_depth_const_dict[catchment_num]
gt_vec = gt_vec*max_depth_const_dict[catchment_num]

In [ ]:
## pred vs gt
from matplotlib.colors import LogNorm

t1 = np.max(pred_vec)
t2 = np.max(gt_vec)
#print(t1)
#print(t2)
max_p = max(t1, t2)
#print(max_p)

plt.hist2d(pred_vec, gt_vec, bins=30, range=((0,max_p),(0,max_p)), cmap='Blues', norm = LogNorm(), cmin=1)
#plt.hist2d(pred_vec, gt_vec, bins=30, cmap='tab20b', norm = LogNorm())
plt.xlabel("Prediction (m)",fontsize=18)
plt.ylabel("GT water depth(m)", fontsize=18)
#plt.title("Scatter plot errors, cmin=10", fontsize=10)

x=np.linspace(0, max_p, 10) 
y = np.linspace(0, max_p, 10) 
plt.plot(y,x,'k-') # identity line

#plt.xlim([2, 6])
#plt.ylim([2, 6])
cb = plt.colorbar()
cb.set_label('counts in bin')
filename = folder_loc + str(h51) + 'scatter_pred_gt_hist.png'
#filename = folder_loc + 'scatter_pred_gt_hist_exp2_0.pdf'
#plt.savefig(filename, dpi=1200)

In [ ]:
import seaborn as sns
import pandas as pd
from conf import max_depth_const_dict
max_depth = max_depth_const_dict[catchment_num]

sns.set_theme(style="darkgrid")

print(pred_vec.shape)
print(gt_vec.shape)
a = [pred_vec, gt_vec]

a = np.array(a).T
a_df = pd.DataFrame(data=a, columns=['pred_vec', 'gt_vec'])
a_df_sorted = a_df.sort_values(by=['gt_vec'])
a_df_sorted.reset_index(drop=True, inplace=True)
a_df_error = 100 * np.abs(a_df_sorted.pred_vec - a_df_sorted.gt_vec)    # predicting in meter, so need to be multiplied by 100

data = np.array([a_df_error[a_df_sorted.gt_vec<0.1],
        a_df_error[(a_df_sorted.gt_vec>0.10) & (a_df_sorted.gt_vec<0.20)],
        a_df_error[(a_df_sorted.gt_vec>0.20) & (a_df_sorted.gt_vec<0.50)],
        a_df_error[(a_df_sorted.gt_vec>0.50) & (a_df_sorted.gt_vec<1)],
        a_df_error[a_df_sorted.gt_vec>1]], dtype="object")
fig, ax = plt.subplots(figsize = [16,8])
ax.set_ylabel('Absolute Error (cm)', fontsize=18)
ax.set_xlabel('Water Depth', fontsize=18)
flierprops = dict(marker='d', markerfacecolor='black', markersize=4, linestyle='none', markeredgecolor='black')
bp = ax.boxplot(data, showfliers=False, patch_artist=True, flierprops=flierprops)                                #'Hide Outlier Points'
ax.set_xticklabels(['0-10cm', '10-20cm', '20-50cm', '50-100cm','>100cm'])

fill_color = ['#ef8a62', '#67a9cf', '#1b9e77', '#ca0020', '#998ec3']
o = 0
for box in bp['boxes']:
    # change outline color
    box.set(color='black', linewidth=1.5)
    # change fill color
    box.set(facecolor = fill_color[o])
    o = o+1
    
for box in bp['medians']:
    # change the median color to black
    box.set(color='black')
    
ax.tick_params(axis='both', labelsize=16)
filename = folder_loc + str(h51) + 'box_plot_' + str(event_num) +'.png'
plt.savefig(filename, dpi=400)
plt.show()

In [ ]:
from evaluation import plot_answer_sample

zoom = [500,600,500,600]
#save_folder = "/scratch2/ml_flood/data/checkpoints/709/cluster/utae_L1/experiment_0/results/c_recons/"
plot_answer_sample(pred_cnn, gt_cnn, mask_cnn, ts=1, zoom=None, show_diff=False, global_scale=True, save_folder = None, model_name = 'cnn')

In [ ]:
zoom = [1500,2000,0,500]
plot_answer_sample(pred_cnn, gt_cnn, mask_cnn, ts=1, zoom=zoom, show_diff=False, global_scale=True, save_folder = None, model_name = 'cnn')

In [ ]:
### count number of 

----

### Prediction with overlapping

In [ ]:
mx = dataset.px - dataset.nx + 1 
my = dataset.py - dataset.ny + 1

indx =  np.arange(0, mx, int(dataset.nx/2))
indy =  np.arange(0, my, int(dataset.ny/2))
    
x, y = np.meshgrid(indx,indy)

patch_x = mx//dataset.nx + 1
patch_y = my//dataset.ny +1 

print(mx, my, dataset.nx)
print('Number of overlapping patches in each direction :', 2 * patch_x, 2 *patch_y)   #9,11


In [ ]:
dataset.inds

In [ ]:
from utils import to_device_eval
    
event_num = 1
target = dataset.peak[event_num]

def create_inputs(dataset, x_p, y_p, xin, rainfall, mask, dem, diff_dem, xout):
        xin, mask, dem, diff_dem, xout = dataset.crop_to_patch(x_p, y_p, xin, mask, dem, diff_dem, xout)
        inputs = dataset.build_inputs(xin, rainfall, mask, dem, diff_dem)
        return inputs, xout
    
def crop_overlapping_patches(y_pred, patch_dim):
    
    cropped = y_pred[10: 
                     int(patch_dim/2) + 10, 
                     10: 
                     int(patch_dim/2) + 10]
    return cropped
    

patch_dim = dataset.nx
inds = dataset.get_all_fix_indexes(non_full=False) 
b = dataset.border_size
xin = dataset.start_ts.clone()
rainfall = dataset.rainfall_events[event_num].clone()
dem = dataset.dem
diff_dem = dataset.diff_dem
mask = dataset.dem_mask
recons_pred_full = torch.zeros(dataset.px - 2*b, dataset.py - 2*b)
recons_gt_full = torch.zeros(dataset.px - 2*b, dataset.py - 2*b)

for inds_count, (x_p, y_p) in enumerate(inds):
    inputs_ts, xout = create_inputs(dataset, x_p, y_p, xin, rainfall, mask, dem, diff_dem, target)
    (data, mask1) = inputs_ts
    data = data.unsqueeze(dim=0)
    data = to_device_eval(data)
    y_pred = model(data)['y_pred'].squeeze().detach().cpu()
    
    data_in = data.squeeze().detach().cpu()
    y_true = xout.squeeze().detach().cpu()   

#     recons_pred_full[x_p + 10:
#                      10 + (int(patch_dim/2))*(inds_count+1),
#                      y_p + 10:
#                      10 + (int(patch_dim/2))*(inds_count+1)] = crop_overlapping_patches(y_pred, patch_dim)
#     recons_gt_full[x_p + 10:
#                      10 + (int(patch_dim/2))*(inds_count+1),
#                      y_p + 10:
#                      10 + (int(patch_dim/2))*(inds_count+1)] = crop_overlapping_patches(y_true, patch_dim)
    
    recons_pred_full[x_p + 10:
                     x_p + 10 + (int(patch_dim/2)),
                     y_p + 10:
                     y_p + 10 + (int(patch_dim/2))] = crop_overlapping_patches(y_pred, patch_dim)
    recons_gt_full[x_p + 10:
                    x_p + 10 + (int(patch_dim/2)),
                    y_p + 10:
                    y_p + 10 + (int(patch_dim/2))] = crop_overlapping_patches(y_true, patch_dim)
#     recons_pred_full[x_p:x_p+patch_dim, y_p:y_p + patch_dim] = y_pred
#     recons_gt_full[x_p:x_p+patch_dim, y_p:y_p + patch_dim] = y_true
#     break



In [ ]:
recons_mask_full = dataset.dem_mask
import matplotlib
ev = 0
for ev in range(1):
    
    pred =  recons_pred_full*recons_mask_full#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
    pred = pred.cpu().detach().numpy()
#     pred = pred [1000:1300, 2000:2300]
    gt = recons_gt_full.cpu().detach().numpy()#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
#     gt = gt[1000:1300, 2000:2300]

    max_v = np.max(pred)
    min_v = np.min(pred)

    if np.max(gt) > max_v:
        max_v = np.max(gt)
    
    l = []
    l.append(gt)
    l.append(pred)

    title = []
    title.append('Ground truth')
    title.append('Prediction')

    i = 0
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,16))
    for ax in axes.flat:
        ax.grid(False)
        ax.title.set_text(title[i])
        im = ax.imshow(l[i], cmap='Blues', norm=matplotlib.colors.LogNorm(vmin=0.0001, vmax=max_v))
        i = i + 1

    fig.subplots_adjust(right=0.8)
    fig.colorbar(im, ax=ax, extend='both', fraction=0.036, pad=0.02)

#     figtitle = "unet, 500 epochs, event=" + str(event_num)
#     plt.suptitle(figtitle, y=0.86)

    filename = folder_loc + str(h51) + 'catch_pred_gt_' + str(event_num)+ '.png'
#     plt.savefig(filename, dpi=1200)

    plt.show()

In [ ]:
recons_mask_full = dataset.dem_mask
import matplotlib
ev = 0
for ev in range(1):
    
    pred =  recons_pred_full*recons_mask_full#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
    pred = pred.cpu().detach().numpy()
#     pred = pred [1000:1300, 2000:2300]
    gt = recons_gt_full.cpu().detach().numpy()#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
#     gt = gt[1000:1300, 2000:2300]

    max_v = np.max(pred)
    min_v = np.min(pred)

    if np.max(gt) > max_v:
        max_v = np.max(gt)
    
    l = []
    l.append(gt)
    l.append(pred)

    title = []
    title.append('Ground truth')
    title.append('Prediction')

    i = 0
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,16))
    for ax in axes.flat:
        ax.grid(False)
        ax.title.set_text(title[i])
        im = ax.imshow(l[i], cmap='Blues', norm=matplotlib.colors.LogNorm(vmin=0.0001, vmax=max_v))
        i = i + 1

    fig.subplots_adjust(right=0.8)
    fig.colorbar(im, ax=ax, extend='both', fraction=0.036, pad=0.02)

#     figtitle = "unet, 500 epochs, event=" + str(event_num)
#     plt.suptitle(figtitle, y=0.86)

    filename = folder_loc + str(h51) + 'catch_pred_gt_' + str(event_num)+ '.png'
#     plt.savefig(filename, dpi=1200)

    plt.show()

In [ ]:
recons_mask_full = dataset.dem_mask
import matplotlib
ev = 0
for ev in range(1):
    
    pred =  recons_pred_full*recons_mask_full#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
    pred = pred.cpu().detach().numpy()
    
    
#     pred = pred [1000:1300, 2000:2300]
    pred = pred [1000:1300, 2000:2300]#[800:1500, 1800:2500]
    gt = recons_gt_full.cpu().detach().numpy()#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
    gt = gt [1000:1300, 2000:2300] #[800:1500, 1800:2500] #[1000:1300, 2000:2300]

    max_v = np.max(pred)
    min_v = np.min(pred)

    if np.max(gt) > max_v:
        max_v = np.max(gt)
    
    l = []
    l.append(gt)
    l.append(pred)

    title = []
    title.append('Ground truth')
    title.append('Prediction')

    i = 0
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,16))
    for ax in axes.flat:
        ax.grid(False)
        ax.title.set_text(title[i])
        im = ax.imshow(l[i], cmap='Blues', norm=matplotlib.colors.LogNorm(vmin=0.0001, vmax=max_v))
        i = i + 1

    fig.subplots_adjust(right=0.8)
    fig.colorbar(im, ax=ax, extend='both', fraction=0.036, pad=0.02)

#     figtitle = "unet, 500 epochs, event=" + str(event_num)
#     plt.suptitle(figtitle, y=0.86)

    filename = folder_loc + str(h51) + 'catch_pred_gt_' + str(event_num)+ '.png'
#     plt.savefig(filename, dpi=1200)

    plt.show()

In [ ]:
recons_mask_full = dataset.dem_mask
import matplotlib
ev = 0
for ev in range(1):
    
    pred =  recons_pred_full*recons_mask_full#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
    pred = pred.cpu().detach().numpy()
    pred = pred [1000:1300, 2000:2300]
    gt = recons_gt_full.cpu().detach().numpy()#[2000:2100, 2000:2100]#[1600:1700, 800:900]#[1800:1900, 900:1000]
    gt = gt[1000:1300, 2000:2300]

    max_v = np.max(pred)
    min_v = np.min(pred)

    if np.max(gt) > max_v:
        max_v = np.max(gt)
    
    l = []
    l.append(gt)
    l.append(pred)

    title = []
    title.append('Ground truth')
    title.append('Prediction')

    i = 0
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,16))
    for ax in axes.flat:
        ax.grid(False)
        ax.title.set_text(title[i])
        im = ax.imshow(l[i], cmap='Blues', norm=matplotlib.colors.LogNorm(vmin=0.0001, vmax=max_v))
        i = i + 1

    fig.subplots_adjust(right=0.8)
    fig.colorbar(im, ax=ax, extend='both', fraction=0.036, pad=0.02)

#     figtitle = "unet, 500 epochs, event=" + str(event_num)
#     plt.suptitle(figtitle, y=0.86)

    filename = folder_loc + str(h51) + 'catch_pred_gt_' + str(event_num)+ '.png'
#     plt.savefig(filename, dpi=1200)

    plt.show()

----